In [1]:
import open3d as o3d
from open3d import visualization
import numpy as np
from pathlib import Path
from mdm_to_openpose import mdm2openpose
from joint_format import *
import os

In [2]:
mdm_path = Path("/home/notingcode/Projects/camera_pose/save_result/results.npy")

test = mdm2openpose(mdm_path)

In [3]:
point_colors = []
joint_pair_idxs = []
for color_rgb in OPENPOSE_JOINT_COLOR.values():
    point_colors.append(color_rgb)
for joint_pair in OPENPOSE_JOINT_PAIRS:
    joint_pair_idxs.append([OPENPOSE_JOINT_MAP[joint_pair[0]],OPENPOSE_JOINT_MAP[joint_pair[1]]])
    
point_colors = np.asarray(point_colors)
point_colors = np.divide(point_colors,255)
joint_pair_idxs = np.asarray(joint_pair_idxs)
joint_pair_colors = np.asarray(OPENPOSE_JOINT_PAIRS_COLOR)
joint_pair_colors = np.divide(joint_pair_colors,255)

In [4]:
# Non-blocking visualization implementation
vis = visualization.Visualizer()
vis.create_window(left=0,top=0,height=1024, width=1024)
vc = vis.get_view_control()

pcd = o3d.geometry.PointCloud()
lineSet = o3d.geometry.LineSet()

render_option = vis.get_render_option()
render_option.background_color = np.asarray([0,0,0])
render_option.point_size = 9
render_option.line_width = 10

for action_idx in range(test.openpose_motion.shape[0]//3):

    entire_motion = test.openpose_motion[action_idx,:,:,:]

    for frame_idx in range(entire_motion.shape[2]//3):
        
        
        pcd.points = o3d.utility.Vector3dVector(entire_motion[:,:,frame_idx])
        pcd.colors = o3d.utility.Vector3dVector(point_colors)
        lineSet.points = o3d.utility.Vector3dVector(entire_motion[:,:,frame_idx])
        lineSet.lines = o3d.utility.Vector2iVector(joint_pair_idxs)
        lineSet.colors = o3d.utility.Vector3dVector(joint_pair_colors)
        center = pcd.get_center()
        vc.set_lookat([center[0],center[1]-0.2,center[2]])
        vc.set_up([0,1,0])
        vc.set_zoom(0.7)
        vc.set_front([0,0,1])
        if frame_idx == 0:
            vis.add_geometry(pcd)
            vis.add_geometry(lineSet)
        else:
            vis.update_geometry(pcd)
            vis.update_geometry(lineSet)
        vis.poll_events()
        vis.update_renderer()
        if frame_idx == 100:
            vis.capture_screen_image(
                os.path.join("/home/notingcode/Projects/camera_pose/images/",f"{action_idx}.png")
            )

vis.destroy_window()

del render_option
del vis
del vc